In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn import metrics
from tqdm import tqdm
import torch
import torch.nn as nn
from efficientnet_pytorch import model as enet
import random
from sklearn.model_selection import StratifiedKFold
import codecs

In [ ]:
def set_seed(seed = 0):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state

random_state = set_seed(2021)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

In [ ]:
import albumentations as A
ttransform = A.Compose([
    
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
])

class ClassificationDataset:
    
    def __init__(self, image_paths, targets): 
        self.image_paths = image_paths
        self.targets = targets

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):      
        image = np.load(self.image_paths[item])
        image = np.vstack(image).transpose((1, 0)).astype(np.float32)[np.newaxis, ]
        # image = ttransform(image = image)["image"][np.newaxis, ]
        # dx = np.random.randint(0, 16)
        # dy = np.random.randint(0, 138)
        # image = image[:, dx:dx+240, dy:dy+1500]

        targets = self.targets[item]
                
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.long),
        }

In [ ]:
df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
print (df.shape)
df['img_path'] = df['id'].apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')

In [ ]:
image = np.load(df['img_path'].iloc[0])
image = np.vstack(image).transpose((1, 0)).astype(np.float32)[np.newaxis, ]
image.shape

In [ ]:
class enetv2(nn.Module):
    def __init__(self, backbone, out_dim):
        super(enetv2, self).__init__()
        self.enet = enet.EfficientNet.from_pretrained(backbone)
        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=3, bias=False)

    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.conv1(x)
        x = self.extract(x)
        x = self.myfc(x)
        return x

In [ ]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a.view(-1, 1)) + (1 - lam) * criterion(pred, y_b.view(-1, 1))

In [ ]:
criterion = nn.BCEWithLogitsLoss()

def train(data_loader, model, optimizer, device, scheduler):
    model.train()
    
    for data in tqdm(data_loader, position=0, leave=True, desc='Training'):
        inputs = data["image"]
        targets = data['targets']
        
        inputs = inputs.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        
        optimizer.zero_grad()
        
        if np.random.randint(1, 10) >= 5:
            mixed_x, y_a, y_b, lam = mixup_data(inputs, targets)
            outputs = model(mixed_x)
            loss = mixup_criterion(criterion, outputs, y_a, y_b, lam)
        else:
            outputs = model(inputs)
            loss = criterion(outputs, targets.view(-1, 1))
        
        # loss = criterion(outputs, targets.view(-1, 1))
        loss.backward()
        optimizer.step()
        scheduler.step()
        
def evaluate(data_loader, model, device):
    model.eval()
    
    final_targets = []
    final_outputs = []
    
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            inputs = data["image"]
            targets = data["targets"]
            inputs = inputs.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            
            output = model(inputs)
            
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(output)
    return final_outputs, final_targets

In [ ]:
baseline_name = 'efficientnet-b0'
models = []
device = "cuda"
epochs = 4
Batch_Size = 16
X = df.img_path.values
Y = df.target.values
skf = StratifiedKFold(n_splits=5)
fold = 0

for train_index, test_index in skf.split(X, Y):
    
    model = enetv2(baseline_name, out_dim=1)
    model.to(device)
    model = nn.DataParallel(model)
    
    train_images, valid_images = X[train_index], X[test_index]
    train_targets, valid_targets = Y[train_index], Y[test_index]

    train_dataset = ClassificationDataset(image_paths=train_images, targets=train_targets)
    valid_dataset = ClassificationDataset(image_paths=valid_images, targets=valid_targets)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=Batch_Size,shuffle=True, num_workers=4)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=Batch_Size,shuffle=False, num_workers=4)

    optimizer = torch.optim.Adam(model.parameters())
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=5e-4, max_lr=1e-3, gamma=0.9, cycle_momentum=False,
                                                  step_size_up=1400,step_size_down=1400, mode="triangular2")
    
    best_roc_auc = 0
    for epoch in range(epochs):
        train(train_loader, model, optimizer, device, scheduler)
        predictions, valid_targets = evaluate(valid_loader, model, device=device)
        roc_auc = metrics.roc_auc_score(valid_targets, predictions)
        print(f"Epoch={epoch}, Valid ROC AUC={roc_auc}")
        
        with codecs.open('log.log', 'a') as up:
            up.write(f"Fold={fold}, Epoch={epoch}, Valid ROC AUC={roc_auc}/{best_roc_auc}\n")

        if roc_auc > best_roc_auc:        
            torch.save(model.state_dict(), baseline_name + '-' + str(fold) + '.pt')
            best_roc_auc = roc_auc
        
    torch.save(model.state_dict(),baseline_name + '-' + str(fold) + '.pt')
    models.append(model)
    fold += 1

In [ ]:
submission = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
submission['img_path'] = submission['id'].apply(lambda x: f'../input/seti-breakthrough-listen/test/{x[0]}/{x}.npy')

In [ ]:
test_dataset = ClassificationDataset(image_paths=submission.img_path.values, targets=submission.target.values)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [ ]:
sig = torch.nn.Sigmoid()
outs = []
for model in models:
    predictions, valid_targets = evaluate(test_loader, model, device=device)
    predictions = np.array(predictions)[:, 0]
    out = sig(torch.from_numpy(predictions))
    out = out.detach().numpy()
    outs.append(out)

In [ ]:
pred = np.mean(np.array(outs), axis=0)

In [ ]:
submission.target = pred
submission.drop(['img_path'], axis=1, inplace=True)
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()